In [1]:
import pandas as pd
import json
import requests
import ast
import yaml

In [2]:
def create_twitter_url():
    handle = "jessicagarson"
    max_results = 100
    mrf = "max_results={}".format(max_results)
    q = "query=from:{}".format(handle)
    url = "https://api.twitter.com/labs/2/tweets/search?{}&{}".format(mrf, q)
    return url

In [3]:
url = create_twitter_url()
print(url)

https://api.twitter.com/labs/2/tweets/search?max_results=100&query=from:jessicagarson


In [4]:
def process_yaml():
    with open("config.yaml") as file:
        return yaml.safe_load(file)

In [5]:
data = process_yaml()

In [6]:
def create_bearer_token(data):
    return data["search_tweets_api"]["bearer_token"]

In [7]:
bearer_token = create_bearer_token(data)

In [8]:
def twitter_auth_and_connect(bearer_token, url):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    response = requests.request("GET", url, headers=headers)
    return response.json()

In [9]:
res_json = twitter_auth_and_connect(bearer_token, url)
print(res_json)
if res_json == {'meta': {'result_count': 0}}:
    print("Check to see if the Twitter handle has Tweeted in the last 7 days")

{'data': [{'id': '1272881032308629506', 'text': '@nomadaisy @kndl I just want to do deals with you'}, {'id': '1272880943687258112', 'text': '@nomadaisy @kndl I live too far away to hang responsibly with y’all 😬😭'}, {'id': '1272711045606408192', 'text': '@Babycastles https://t.co/Yfj8SJAnpG'}, {'id': '1272390182231330816', 'text': '@replylord Haha, I broke a glass in your honor today and all so I think I do read your Tweets'}, {'id': '1271810907274915840', 'text': '@replylord I like that I’m the only like here.'}, {'id': '1271435152183476225', 'text': '@Arfness @ChicagoPython @codewithbri @WeCodeDreams @agfors The video seems to be available https://t.co/GojUGdulkP'}, {'id': '1271111488024064001', 'text': 'RT @TwitterDev: Tune in tonight and watch as @jessicagarson takes us through running your favorite Python package in R. 🍿\n\nLearn how to use…'}, {'id': '1270794941892046848', 'text': 'RT @ChicagoPython: Chicago Python will be live-streaming tmrw night!\n\nOur talks:\n- How to run you

In [10]:
def lang_data_shape(res_json):
    data_only = res_json["data"]
    doc_start = '"documents": {}'.format(data_only)
    str_json = "{" + doc_start + "}"
    dump_doc = json.dumps(str_json)
    doc = json.loads(dump_doc)
    return ast.literal_eval(doc)

In [11]:
documents = lang_data_shape(res_json)
print(documents)

{'documents': [{'id': '1272881032308629506', 'text': '@nomadaisy @kndl I just want to do deals with you'}, {'id': '1272880943687258112', 'text': '@nomadaisy @kndl I live too far away to hang responsibly with y’all 😬😭'}, {'id': '1272711045606408192', 'text': '@Babycastles https://t.co/Yfj8SJAnpG'}, {'id': '1272390182231330816', 'text': '@replylord Haha, I broke a glass in your honor today and all so I think I do read your Tweets'}, {'id': '1271810907274915840', 'text': '@replylord I like that I’m the only like here.'}, {'id': '1271435152183476225', 'text': '@Arfness @ChicagoPython @codewithbri @WeCodeDreams @agfors The video seems to be available https://t.co/GojUGdulkP'}, {'id': '1271111488024064001', 'text': 'RT @TwitterDev: Tune in tonight and watch as @jessicagarson takes us through running your favorite Python package in R. 🍿\n\nLearn how to use…'}, {'id': '1270794941892046848', 'text': 'RT @ChicagoPython: Chicago Python will be live-streaming tmrw night!\n\nOur talks:\n- How to ru

In [12]:
def connect_to_azure(data):
    azure_url = "https://week.cognitiveservices.azure.com/"
    language_api_url = "{}text/analytics/v2.1/languages".format(azure_url)
    sentiment_url = "{}text/analytics/v2.1/sentiment".format(azure_url)
    subscription_key = data["azure"]["subscription_key"]
    return language_api_url, sentiment_url, subscription_key

In [13]:
language_api_url, sentiment_url, subscription_key = connect_to_azure(data)

In [14]:
def azure_header(subscription_key):
    return {"Ocp-Apim-Subscription-Key": subscription_key}

In [15]:
headers = azure_header(subscription_key)

In [16]:
def generate_languages(headers, language_api_url, documents):
    response = requests.post(language_api_url, headers=headers, json=documents)
    return response.json()

In [17]:
with_languages = generate_languages(headers, language_api_url, documents)
print(with_languages)

{'documents': [{'id': '1272881032308629506', 'detectedLanguages': [{'name': 'English', 'iso6391Name': 'en', 'score': 1.0}]}, {'id': '1272880943687258112', 'detectedLanguages': [{'name': 'English', 'iso6391Name': 'en', 'score': 1.0}]}, {'id': '1272711045606408192', 'detectedLanguages': [{'name': 'English', 'iso6391Name': 'en', 'score': 1.0}]}, {'id': '1272390182231330816', 'detectedLanguages': [{'name': 'English', 'iso6391Name': 'en', 'score': 1.0}]}, {'id': '1271810907274915840', 'detectedLanguages': [{'name': 'English', 'iso6391Name': 'en', 'score': 1.0}]}, {'id': '1271435152183476225', 'detectedLanguages': [{'name': 'English', 'iso6391Name': 'en', 'score': 1.0}]}, {'id': '1271111488024064001', 'detectedLanguages': [{'name': 'English', 'iso6391Name': 'en', 'score': 1.0}]}, {'id': '1270794941892046848', 'detectedLanguages': [{'name': 'English', 'iso6391Name': 'en', 'score': 1.0}]}, {'id': '1270485552488427521', 'detectedLanguages': [{'name': 'English', 'iso6391Name': 'en', 'score': 1.0

In [18]:
def combine_lang_data(documents, with_languages):
    langs = pd.DataFrame(with_languages["documents"])
    lang_iso = [x.get("iso6391Name")
                for d in langs.detectedLanguages if d for x in d]
    data_only = documents["documents"]
    tweet_data = pd.DataFrame(data_only)
    tweet_data.insert(2, "language", lang_iso, True)
    json_lines = tweet_data.to_json(orient="records")
    return json_lines

In [19]:
json_lines = combine_lang_data(documents, with_languages)
print(json_lines)

[{"id":"1272881032308629506","text":"@nomadaisy @kndl I just want to do deals with you","language":"en"},{"id":"1272880943687258112","text":"@nomadaisy @kndl I live too far away to hang responsibly with y\u2019all \ud83d\ude2c\ud83d\ude2d","language":"en"},{"id":"1272711045606408192","text":"@Babycastles https:\/\/t.co\/Yfj8SJAnpG","language":"en"},{"id":"1272390182231330816","text":"@replylord Haha, I broke a glass in your honor today and all so I think I do read your Tweets","language":"en"},{"id":"1271810907274915840","text":"@replylord I like that I\u2019m the only like here.","language":"en"},{"id":"1271435152183476225","text":"@Arfness @ChicagoPython @codewithbri @WeCodeDreams @agfors The video seems to be available https:\/\/t.co\/GojUGdulkP","language":"en"},{"id":"1271111488024064001","text":"RT @TwitterDev: Tune in tonight and watch as @jessicagarson takes us through running your favorite Python package in R. \ud83c\udf7f\n\nLearn how to use\u2026","language":"en"},{"id":"127

In [20]:
def add_document_format(json_lines):
    docu_format = '"' + "documents" + '"'
    json_docu_format = "{}:{}".format(docu_format, json_lines)
    docu_align = "{" + json_docu_format + "}"
    jd_align = json.dumps(docu_align)
    jl_align = json.loads(jd_align)
    return ast.literal_eval(jl_align)

In [21]:
document_format = add_document_format(json_lines)
print(document_format)

{'documents': [{'id': '1272881032308629506', 'text': '@nomadaisy @kndl I just want to do deals with you', 'language': 'en'}, {'id': '1272880943687258112', 'text': '@nomadaisy @kndl I live too far away to hang responsibly with y’all \ud83d\ude2c\ud83d\ude2d', 'language': 'en'}, {'id': '1272711045606408192', 'text': '@Babycastles https:\\/\\/t.co\\/Yfj8SJAnpG', 'language': 'en'}, {'id': '1272390182231330816', 'text': '@replylord Haha, I broke a glass in your honor today and all so I think I do read your Tweets', 'language': 'en'}, {'id': '1271810907274915840', 'text': '@replylord I like that I’m the only like here.', 'language': 'en'}, {'id': '1271435152183476225', 'text': '@Arfness @ChicagoPython @codewithbri @WeCodeDreams @agfors The video seems to be available https:\\/\\/t.co\\/GojUGdulkP', 'language': 'en'}, {'id': '1271111488024064001', 'text': 'RT @TwitterDev: Tune in tonight and watch as @jessicagarson takes us through running your favorite Python package in R. \ud83c\udf7f\n\nLe

In [22]:
def sentiment_scores(headers, sentiment_url, document_format):
    response = requests.post(
        sentiment_url, headers=headers, json=document_format)
    return response.json()

In [29]:
sentiments = sentiment_scores(headers, sentiment_url, document_format)
print(sentiments)

{'documents': [{'id': '1272881032308629506', 'score': 0.18426942825317383}, {'id': '1272880943687258112', 'score': 0.0031259357929229736}, {'id': '1272711045606408192', 'score': 0.7015109062194824}, {'id': '1272390182231330816', 'score': 0.8754926323890686}, {'id': '1271810907274915840', 'score': 0.19140595197677612}, {'id': '1271435152183476225', 'score': 0.7853382229804993}, {'id': '1271111488024064001', 'score': 0.7884223461151123}, {'id': '1270794941892046848', 'score': 0.8826596736907959}, {'id': '1270485552488427521', 'score': 0.8784275054931641}], 'errors': []}


In [30]:
def mean_score(sentiments):
    sentiment_df = pd.DataFrame(sentiments["documents"])
    return sentiment_df["score"].mean()

In [31]:
week_score = mean_score(sentiments)
print(week_score)

0.5878502892123328


In [32]:
def week_logic(week_score):
    if week_score > 0.75 or week_score == 0.75:
        print("You had a positive week")
    elif week_score > 0.45 or week_score == 0.45:
        print("You had a neutral week")
    else:
        print("You had a negative week, I hope it gets better")

In [33]:
week_logic(week_score)

You had a neutral week
